In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from dmba import adjusted_r2_score, AIC_score, BIC_score
from dmba import regressionSummary, exhaustive_search
# Load the dataset
boston_df = pd.read_csv('/Users/jayasreemuchamari/Documents/Jayasreemasterstudymaterial/BAN620/Casestudy1/BostonHousing.csv')
#Display dimensions
print("Dataset dimensions:", boston_df.shape)
print(boston_df.head())

no display found. Using non-interactive Agg backend
Dataset dimensions: (506, 14)
     CRIME  ZONE  INDUST CHAR RIV  NIT OXIDE  ROOMS   AGE  DISTANCE  RADIAL  \
0  0.00632  18.0    2.31        N      0.538  6.575  65.2    4.0900       1   
1  0.02731   0.0    7.07        N      0.469  6.421  78.9    4.9671       2   
2  0.02729   0.0    7.07        N      0.469  7.185  61.1    4.9671       2   
3  0.03237   0.0    2.18        N      0.458  6.998  45.8    6.0622       3   
4  0.06905   0.0    2.18        N      0.458  7.147  54.2    6.0622       3   

   TAX  ST RATIO  LOW STAT  MVALUE C MVALUE  
0  296      15.3      4.98    24.0       No  
1  242      17.8      9.14    21.6       No  
2  242      17.8      4.03    34.7      Yes  
3  222      18.7      2.94    33.4      Yes  
4  222      18.7      5.33    36.2      Yes  


In [2]:
#getting columns
print(boston_df.columns)
# Rename columns with one-word titles
boston_df.columns = [col.replace(" ", "_") for col in boston_df.columns]
print("Modified column names:", boston_df.columns.tolist())

Index(['CRIME', 'ZONE', 'INDUST', 'CHAR RIV', 'NIT OXIDE', 'ROOMS', 'AGE',
       'DISTANCE', 'RADIAL', 'TAX', 'ST RATIO', 'LOW STAT', 'MVALUE',
       'C MVALUE'],
      dtype='object')
Modified column names: ['CRIME', 'ZONE', 'INDUST', 'CHAR_RIV', 'NIT_OXIDE', 'ROOMS', 'AGE', 'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'MVALUE', 'C_MVALUE']


In [3]:
print(boston_df.dtypes)

CRIME        float64
ZONE         float64
INDUST       float64
CHAR_RIV      object
NIT_OXIDE    float64
ROOMS        float64
AGE          float64
DISTANCE     float64
RADIAL         int64
TAX            int64
ST_RATIO     float64
LOW_STAT     float64
MVALUE       float64
C_MVALUE      object
dtype: object


In [4]:
boston_df[['CHAR_RIV', 'C_MVALUE']].head(10)


,CHAR_RIV,C_MVALUE
0,N,No
1,N,No
2,N,Yes
3,N,Yes
4,N,Yes
5,N,No
6,N,No
7,N,No
8,N,No
9,N,No


In [5]:
# Convert categorical variable CHAR_RIV to dummy variable
boston_df['CHAR_RIV'] = boston_df['CHAR_RIV'].map({'Y': 1, 'N': 0})
boston_df['C_MVALUE'] = boston_df['C_MVALUE'].map({'Yes': 1, 'No': 0})
# Display data types
print("Data types:\n", boston_df.dtypes)
print(boston_df.head())

Data types:
 CRIME        float64
ZONE         float64
INDUST       float64
CHAR_RIV       int64
NIT_OXIDE    float64
ROOMS        float64
AGE          float64
DISTANCE     float64
RADIAL         int64
TAX            int64
ST_RATIO     float64
LOW_STAT     float64
MVALUE       float64
C_MVALUE       int64
dtype: object
     CRIME  ZONE  INDUST  CHAR_RIV  NIT_OXIDE  ROOMS   AGE  DISTANCE  RADIAL  \
0  0.00632  18.0    2.31         0      0.538  6.575  65.2    4.0900       1   
1  0.02731   0.0    7.07         0      0.469  6.421  78.9    4.9671       2   
2  0.02729   0.0    7.07         0      0.469  7.185  61.1    4.9671       2   
3  0.03237   0.0    2.18         0      0.458  6.998  45.8    6.0622       3   
4  0.06905   0.0    2.18         0      0.458  7.147  54.2    6.0622       3   

   TAX  ST_RATIO  LOW_STAT  MVALUE  C_MVALUE  
0  296      15.3      4.98    24.0         0  
1  242      17.8      9.14    21.6         0  
2  242      17.8      4.03    34.7         1  
3  222    

In [6]:
boston_df[['CHAR_RIV', 'C_MVALUE']].head(10)


,CHAR_RIV,C_MVALUE
0,0,0
1,0,0
2,0,1
3,0,1
4,0,1
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [7]:
"""
d. Display in Python the descriptive statistics for all columns in the modified boston_df data
frame (after converting to one-word titles and dummy variables). Check if there are
2. 3. missing records (values) in the columns. Present the table with descriptive statistics in
your report, and comment about the missing values. You don’t need to comment on the
values of outliers (min/max) or their extreme values.
"""
np.round(boston_df.describe(),2)


,CRIME,ZONE,INDUST,CHAR_RIV,NIT_OXIDE,ROOMS,AGE,DISTANCE,RADIAL,TAX,ST_RATIO,LOW_STAT,MVALUE,C_MVALUE
count,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00,506.00
mean,3.61,11.36,11.14,0.07,0.55,6.28,68.57,3.80,9.55,408.24,18.46,12.65,22.53,0.17
std,8.60,23.32,6.86,0.25,0.12,0.70,28.15,2.11,8.71,168.54,2.16,7.14,9.20,0.37
min,0.01,0.00,0.46,0.00,0.38,3.56,2.90,1.13,1.00,187.00,12.60,1.73,5.00,0.00
25%,0.08,0.00,5.19,0.00,0.45,5.89,45.02,2.10,4.00,279.00,17.40,6.95,17.02,0.00
50%,0.26,0.00,9.69,0.00,0.54,6.21,77.50,3.21,5.00,330.00,19.05,11.36,21.20,0.00
75%,3.68,12.50,18.10,0.00,0.62,6.62,94.07,5.19,24.00,666.00,20.20,16.96,25.00,0.00
max,88.98,100.00,27.74,1.00,0.87,8.78,100.00,12.13,24.00,711.00,22.00,37.97,50.00,1.00


In [8]:
# Check for missing values
print("Missing values:\n", boston_df.isnull().sum())

Missing values:
 CRIME        0
ZONE         0
INDUST       0
CHAR_RIV     0
NIT_OXIDE    0
ROOMS        0
AGE          0
DISTANCE     0
RADIAL       0
TAX          0
ST_RATIO     0
LOW_STAT     0
MVALUE       0
C_MVALUE     0
dtype: int64


In [9]:
"""
2. Develop multiple linear regression with all 13 predictors.
a. Develop in Python outcome and predictor variables, partition the data set (75% for
training and 25% for validation partitions), and train the multiple linear regression model
using LinearRegression() with the training data set. Identify and display in Python
intercept and regression coefficients of this model. Provide these coefficients in your
report and present the mathematical equation of this linear regression model.
"""
# Define predictors and outcome variable
predictors = ['CRIME', 'ZONE', 'INDUST', 'CHAR_RIV', 'NIT_OXIDE', 'ROOMS', 'AGE',
              'DISTANCE', 'RADIAL', 'TAX', 'ST_RATIO', 'LOW_STAT', 'C_MVALUE']
outcome = 'MVALUE'
# Identify X and y variables for regression and partition data
X = boston_df[predictors]
y = boston_df[outcome]
# Split data into training (75%) and validation (25%)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)
# Train multiple linear regression model for training set
boston_model = LinearRegression()
boston_model.fit(X_train, y_train)
# Extract intercept and coefficients
intercept = np.round(boston_model.intercept_, 2)  # Round to 2 decimals
coefficients = dict(zip(X.columns, np.round(boston_model.coef_, 2)))  # Round coefficients
# Display intercept and regression coefficients. Round
# them to 2 decimals.
print('Regression Model for Boston Training Set')
print()
print('Intercept: ', np.round(boston_model.intercept_, 2)) # coefficient Bo
print(pd.DataFrame({'Predictor': X.columns, 'Coefficient': np.round(boston_model.coef_, 2)}))
# Construct the regression equation
equation = f"MVALUE = {intercept:.2f}"
for feature, coef in coefficients.items():
    equation += f" + ({coef:.2f} * {feature})"
print("Regression Equation:\n", equation)

Regression Model for Boston Training Set

Intercept:  41.08
    Predictor  Coefficient
0       CRIME        -0.14
1        ZONE        -0.03
2      INDUST         0.13
3    CHAR_RIV         2.56
4   NIT_OXIDE       -14.29
5       ROOMS         1.19
6         AGE        -0.01
7    DISTANCE        -0.61
8      RADIAL         0.15
9         TAX        -0.01
10   ST_RATIO        -0.55
11   LOW_STAT        -0.47
12   C_MVALUE        12.15
Regression Equation:
 MVALUE = 41.08 + (-0.14 * CRIME) + (-0.03 * ZONE) + (0.13 * INDUST) + (2.56 * CHAR_RIV) + (-14.29 * NIT_OXIDE) + (1.19 * ROOMS) + (-0.01 * AGE) + (-0.61 * DISTANCE) + (0.15 * RADIAL) + (-0.01 * TAX) + (-0.55 * ST_RATIO) + (-0.47 * LOW_STAT) + (12.15 * C_MVALUE)


""""
b. Using the multiple regression model, identify in Python predictions for validation and
training predictors (valid_X and train_X). Based on these predictions, identify and display
in Python R2 and adjusted R2 performance measures for training and validation partitions.
Present and compare these performance measures in your report and explain if there is
a possibility of overfitting."
"""""


In [10]:
# Make predictions
train_pred = boston_model.predict(X_train)
valid_pred = boston_model.predict(X_valid)
# Create prediction performance measures for training set.
r2_train= round(r2_score(y_train, train_pred),3)
adj_r2_train= round(adjusted_r2_score(y_train, train_pred, boston_model),3)  # Use train_pred for adjusted R2
aic = round(AIC_score(y_train, train_pred, boston_model),2)
bic = round(BIC_score(y_train, train_pred, boston_model),2)
# Display prediction performance measures for training set.
print('Prediction Performance Measures for Training Set')
print('r2 : ', r2_train)
print('Adjusted r2 : ', adj_r2_train)
print('AIC_valid: ', aic)
print('BIC_valid: ', bic)
print() 
# Create prediction performance measures for validation set.
r2_valid = round(r2_score(y_valid, valid_pred), 3)  # Use y_test and test_pred for validation
adj_r2_valid = round(adjusted_r2_score(y_valid, valid_pred, boston_model),3) # Use test_pred for adjusted R2
aic_valid = round(AIC_score(y_valid, valid_pred, boston_model), 2)
bic_valid = round(BIC_score(y_valid, valid_pred, boston_model),2)
# Display prediction performance measures for validation set.
print('Prediction Performance Measures for Validation Set')
print('r2 : ', r2_valid)
print('Adjusted r2 : ', adj_r2_valid)
print('AIC_valid: ', aic_valid)
print('BIC_valid: ', bic_valid)



Prediction Performance Measures for Training Set
r2 :  0.849
Adjusted r2 :  0.843
AIC_valid:  2090.06
BIC_valid:  2149.12

Prediction Performance Measures for Validation Set
r2 :  0.799
Adjusted r2 :  0.776
AIC_valid:  726.47
BIC_valid:  769.13


c. Identify and display in Python the common accuracy measures for training and validation
data set (predictions). Provide and compare these accuracy measures in your report and
assess again a possibility of overfitting.

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
def regressionSummary(y_true, y_pred):
    """Compute and display regression accuracy metrics."""
    mse = mean_squared_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)  # MAPE
    me = np.mean(y_true - y_pred)  # Mean Error (ME)
    mpe = np.mean((y_true - y_pred) / y_true) * 100  # Mean Percentage Error (MPE)

    print(f"Mean Error (ME): {me:.3f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")
    print(f"Mean Absolute Error (MAE): {mae:.3f}")
    print(f"Mean Percentage Error (MPE): {mpe:.2f}%")
    print(f"Mean Absolute Percentage Error (MAPE): {mape * 100:.2f}%")
    print("-" * 50)

# Display accuracy measures for training set
print('Accuracy Measures for Training Set - All Variables')
regressionSummary(y_train, train_pred)

# Display accuracy measures for validation set
print('Accuracy Measures for Validation Set - All Variables')
regressionSummary(y_valid, valid_pred)


Accuracy Measures for Training Set - All Variables
Mean Error (ME): -0.000
Root Mean Squared Error (RMSE): 3.665
Mean Absolute Error (MAE): 2.724
Mean Percentage Error (MPE): -2.75%
Mean Absolute Percentage Error (MAPE): 13.37%
--------------------------------------------------
Accuracy Measures for Validation Set - All Variables
Mean Error (ME): 0.201
Root Mean Squared Error (RMSE): 3.755
Mean Absolute Error (MAE): 2.611
Mean Percentage Error (MPE): -1.16%
Mean Absolute Percentage Error (MAPE): 13.24%
--------------------------------------------------


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Executive Search

In [12]:
# Define train_model() function used in Executive Search
# algorithm with executive_search() function. 
#X_train, X_valid, y_train, y_valid
def train_model(variables):
    model = LinearRegression()
    model.fit(X_train[variables], y_train)
    return model

# Define score_model() function used in Executive Search
# algorithm with executive_search() function. 
def score_model(model, variables):
    pred_y = model.predict(X_train[variables])
    # Apply negative sign as score is optimized to be 
    # as low as possible in exhaustive_search() function.
    return -adjusted_r2_score(y_train, pred_y, model)

# Create allVariables object with predcitors in X_train,
# i.e., training data set, with a11 predictor columns and 
allVariables = X_train.columns

# The exhaustive_search() function consists of 3 arguments:
# - allVariables - list of all variables in training data set,
# - train_model() function that creates a model for a specific 
#    combination of variables,
# - score_model() function that score the model performance using
#     adjusted_r2.
results = exhaustive_search(allVariables, train_model, score_model)

# Create data[] loop process to identify and append the best model 
# for each combination of 1, 2, 3, ..., 13 variables with their 
# respective number of variables (n), adjusted R_squared (r2adj) and
# AIC. 

data = []
for result in results:
    model = result['model']
    variables = result['variables']
    AIC = AIC_score(y_train, model.predict(X_train[variables]), model)
    d = {'n': result['n'], 'r2adj': -result['score'], 'AIC': AIC}
    d.update({var: var in result['variables'] for var in allVariables})
    data.append(d)   

# Define the width of output presentation to be wider
# to display results in two rows 
pd.set_option('display.width', 100)

# Display the Exhaustive Search results.
print(pd.DataFrame(data, columns=('n', 'r2adj', 'AIC') + tuple(sorted(allVariables))))

# Reset the output width to the default. 
pd.reset_option('display.width')


     n     r2adj          AIC    AGE  CHAR_RIV  CRIME  C_MVALUE  DISTANCE  INDUST  LOW_STAT  \
0    1  0.636334  2397.020705  False     False  False      True     False   False     False   
1    2  0.799565  2172.226282  False     False  False      True     False   False      True   
2    3  0.810608  2151.739228  False     False   True      True     False   False      True   
3    4  0.818310  2136.992806  False     False   True      True      True   False      True   
4    5  0.824925  2123.920534  False     False   True      True     False   False      True   
5    6  0.829698  2114.428164  False      True   True      True     False   False      True   
6    7  0.834219  2105.209673  False      True   True      True      True   False      True   
7    8  0.837615  2098.342562  False      True   True      True      True   False      True   
8    9  0.840000  2093.710264  False      True   True      True      True   False      True   
9   10  0.840991  2092.327285  False      True   T

Linear Regression Model by Executive Search

In [ ]:

# Select the 12-variable subset from row 11 from exhastive search
selected_vars = ['CHAR_RIV', 'CRIME', 'C_MVALUE', 'DISTANCE', 'INDUST', 
                 'LOW_STAT', 'NIT_OXIDE', 'RADIAL','ROOMS','ST_RATIO','TAX','ZONE']
y_column = 'MVALUE'
X = boston_df[selected_vars]
y = boston_df[y_column]
# Split into training (75%) and validation (25%) sets
train_X_new, valid_X_new, train_y_new, valid_y_new = train_test_split(X, y, test_size=0.25, random_state=42)
# Create the linear regression model and fit it to the training data
boston_new = LinearRegression()
boston_new.fit(train_X_new, train_y_new)
# Display intercept and regression coefficients (rounded to 2 decimals)
print('Regression Model for Training Set Using Exhaustive Search')
print()
print('Intercept: ', np.round(boston_new.intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns, 'Coefficient': np.round(boston_new.coef_, 2)}))
# Display the regression equation
equation = f"MVALUE = {boston_new.intercept_:.2f}"
for var, coef in zip(selected_vars, boston_new.coef_):
    equation += f" + ({coef:.2f})*{var}"
print("\nRegression Equation:")
print(equation)

Regression Model for Training Set Using Exhaustive Search

Intercept:  41.31
    Predictor  Coefficient
0    CHAR_RIV         2.54
1       CRIME        -0.14
2    C_MVALUE        12.11
3    DISTANCE        -0.54
4      INDUST         0.12
5    LOW_STAT        -0.49
6   NIT_OXIDE       -15.36
7      RADIAL         0.16
8       ROOMS         1.11
9    ST_RATIO        -0.56
10        TAX        -0.01
11       ZONE        -0.03

Regression Equation:
MVALUE = 41.31 + (2.54)*CHAR_RIV + (-0.14)*CRIME + (12.11)*C_MVALUE + (-0.54)*DISTANCE + (0.12)*INDUST + (-0.49)*LOW_STAT + (-15.36)*NIT_OXIDE + (0.16)*RADIAL + (1.11)*ROOMS + (-0.56)*ST_RATIO + (-0.01)*TAX + (-0.03)*ZONE


In [14]:
y_pred_new = boston_new.predict(valid_X_new)
# Compute performance measures for reduced predictors
r2 = r2_score(valid_y_new, y_pred_new)
n = len(valid_y_new)
p = len(selected_vars)
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
rmse = np.sqrt(mean_squared_error(valid_y_new, y_pred_new))
mae = mean_absolute_error(valid_y_new, y_pred_new)
# Compute additional error metrics
mean_error = np.mean(y_pred_new - valid_y_new)  # Mean Error (ME)
mpe = np.mean((valid_y_new - y_pred_new) / valid_y_new) * 100  # Mean Percentage Error (MPE)
mape = np.mean(np.abs((valid_y_new - y_pred_new) / valid_y_new)) * 100  # Mean Absolute Percentage Error (MAPE)
# Compute AIC and BIC
sse = np.sum((valid_y_new - y_pred_new) ** 2)
aic = n * np.log(sse / n) + 2 * (p + 1)
bic = n * np.log(sse / n) + np.log(n) * (p + 1)
# Output the performance measures
print("Accuracy Measures for Validation Set Using Exhaustive Search")
print(f"R-squared: {r2}")
print(f"Adjusted R-squared: {adj_r2}")
print(f"AIC: {aic}")
print(f"BIC: {bic}")
print()
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"Mean Error (ME): {mean_error}")
print(f"Mean Percentage Error (MPE): {mpe}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}")



Accuracy Measures for Validation Set Using Exhaustive Search
R-squared: 0.8017045229748152
Adjusted R-squared: 0.780831314866901
AIC: 360.1225068758537
BIC: 397.09693899981534

RMSE: 3.7264011935965833
MAE: 2.606832004828255
Mean Error (ME): -0.21327096530498507
Mean Percentage Error (MPE): -1.1974377148266186
Mean Absolute Percentage Error (MAPE): 13.270491420493913


"""b. Use the Forward Selection algorithm in Python to identify the best predictors. Based on
these predictors and 75%-25% partition of the data set, train a new multiple linear
regression model. Provide the same results in your report as discussed in 3a. Also, explain
the differences (if any exists) between the best predictors (number and specific predictors
used) in the models in 3a and 3b."""



Forward Selection

In [16]:
def forward_selection(features, train_model, score_model, verbose=False):
    remaining = list(features)
    selected = []
    current_score, best_new_score = float('inf'), float('inf')
    best_model = None

    while remaining:
        scores = []
        for candidate in remaining:
            test_vars = selected + [candidate]
            model = train_model(test_vars)
            score = score_model(model, test_vars)
            scores.append((score, candidate, model))

        scores.sort()
        best_new_score, best_candidate, candidate_model = scores[0]

        if verbose:
            print(f"Evaluated {best_candidate} → AIC: {best_new_score:.2f}")

        if best_new_score < current_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
            best_model = candidate_model
        else:
            break

    return best_model, selected
best_model_fs, best_variables_fs = forward_selection(
    X_train.columns, train_model, score_model, verbose=True
)

print('\nBest Variables from Forward Selection Algorithm:')
print(best_variables_fs)


Evaluated C_MVALUE → AIC: 1319.47
Evaluated LOW_STAT → AIC: 1094.67
Evaluated CRIME → AIC: 1074.18
Evaluated DISTANCE → AIC: 1059.44
Evaluated CHAR_RIV → AIC: 1050.04
Evaluated NIT_OXIDE → AIC: 1040.89
Evaluated ST_RATIO → AIC: 1027.65
Evaluated ROOMS → AIC: 1020.79
Evaluated ZONE → AIC: 1016.15
Evaluated INDUST → AIC: 1014.77
Evaluated RADIAL → AIC: 1014.51
Evaluated TAX → AIC: 1012.09
Evaluated AGE → AIC: 1012.50

Best Variables from Forward Selection Algorithm:
['C_MVALUE', 'LOW_STAT', 'CRIME', 'DISTANCE', 'CHAR_RIV', 'NIT_OXIDE', 'ST_RATIO', 'ROOMS', 'ZONE', 'INDUST', 'RADIAL', 'TAX']


Linear Regression model by Forword Selection

In [17]:
# Train a new model using the best predictors identified by Forward Selection
fs_values = ['C_MVALUE', 'LOW_STAT', 'CRIME', 'DISTANCE', 'CHAR_RIV', 'NIT_OXIDE', 'ST_RATIO', 'ROOMS', 'ZONE', 'INDUST', 'RADIAL', 'TAX']
y_column = 'MVALUE'
# Split into training (75%) and validation (25%) sets
X = boston_df[fs_values]
y = boston_df[y_column]
train_X_fs, valid_X_fs, train_y_fs, valid_y_fs = train_test_split(X, y, test_size=0.25, random_state=42)
boston_new_forward = LinearRegression()
boston_new_forward.fit(train_X_fs[best_variables_fs], train_y_fs)
print('Regression Model for Training Set Using Forward Selection')
print()
print('Intercept ', np.round(boston_new_forward .intercept_, 2))
print(pd.DataFrame({'Predictor': X.columns,
            'Coefficient': np.round(boston_new_forward .coef_, 2)}))

# Display the regression equation
equation = f"MVALUE = {boston_new_forward.intercept_:.2f}"
for var, coef in zip(best_variables_fs, boston_new_forward.coef_):
    equation += f" + ({coef:.2f})*{var}"

print("\nRegression Equation:")
print(equation)


Regression Model for Training Set Using Forward Selection

Intercept  41.31
    Predictor  Coefficient
0    C_MVALUE        12.11
1    LOW_STAT        -0.49
2       CRIME        -0.14
3    DISTANCE        -0.54
4    CHAR_RIV         2.54
5   NIT_OXIDE       -15.36
6    ST_RATIO        -0.56
7       ROOMS         1.11
8        ZONE        -0.03
9      INDUST         0.12
10     RADIAL         0.16
11        TAX        -0.01

Regression Equation:
MVALUE = 41.31 + (12.11)*C_MVALUE + (-0.49)*LOW_STAT + (-0.14)*CRIME + (-0.54)*DISTANCE + (2.54)*CHAR_RIV + (-15.36)*NIT_OXIDE + (-0.56)*ST_RATIO + (1.11)*ROOMS + (-0.03)*ZONE + (0.12)*INDUST + (0.16)*RADIAL + (-0.01)*TAX


In [84]:
# Predict on the validation set
y_pred_new_forward = boston_new_forward.predict(valid_X_fs[best_variables_fs])
# Compute performance measures
r2 = r2_score(valid_y_fs, y_pred_new_forward)
n = len(valid_y_fs)
p = len(best_variables_fs)
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
rmse = np.sqrt(mean_squared_error(valid_y_fs, y_pred_new_forward))
mae = mean_absolute_error(valid_y_fs, y_pred_new_forward)

# Compute additional error metrics
mean_error = np.mean(y_pred_new_forward - valid_y_fs)  # Mean Error (ME)
mpe = np.mean((valid_y_fs - y_pred_new_forward) / valid_y_fs) * 100  # Mean Percentage Error (MPE)
mape = np.mean(np.abs((valid_y_fs - y_pred_new_forward) / valid_y_fs)) * 100  # Mean Absolute Percentage Error (MAPE)

# Compute AIC and BIC
sse = np.sum((valid_y_fs - y_pred_new_forward) ** 2)
aic = n * np.log(sse / n) + 2 * (p + 1)
bic = n * np.log(sse / n) + np.log(n) * (p + 1)

# Output the performance measures
print(f"R-squared: {r2}")
print(f"Adjusted R-squared: {adj_r2}")
print(f"AIC: {aic}")
print(f"BIC: {bic}")
print()
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"Mean Error (ME): {mean_error}")
print(f"Mean Percentage Error (MPE): {mpe}")
print(f"Mean Absolute Percentage Error (MAPE): {mape}")



R-squared: 0.801704522974809
Adjusted R-squared: 0.7808313148668942
AIC: 360.12250687585765
BIC: 397.0969389998193

RMSE: 3.726401193596642
MAE: 2.6068320048283216
Mean Error (ME): -0.21327096530496892
Mean Percentage Error (MPE): -1.1974377148268822
Mean Absolute Percentage Error (MAPE): 13.270491420494263
